In [10]:

# Cell 1: Imports, Mixed Precision, GPU Setup, Configuration (PyTorch Version)

import os, glob, ctypes, math
import h5py, numpy as np, pandas as pd, matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler

from tqdm import tqdm

from sklearn.metrics import (
    confusion_matrix, precision_score, recall_score,
    f1_score, accuracy_score, mean_squared_error, mean_absolute_error
)

# Mixed precision utilities
scaler = GradScaler()

# DLL fix for Windows HDF5/zlib
try:
    dll = os.path.join(os.environ.get('CONDA_PREFIX',''), 'Library', 'bin', 'zlibwapi.dll')
    ctypes.CDLL(dll)
    print("Loaded zlibwapi.dll")
except Exception:
    print("Could not load zlibwapi.dll")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)
torch.backends.cuda.matmul.allow_tf32 = True


Loaded zlibwapi.dll
Using cuda


C:\Users\dhanu\AppData\Local\Temp\ipykernel_22596\344887725.py:19: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


## Data‑Loading & Generator Functions (unchanged except for PyTorch tensors)

In [11]:

# Cell 2: Data Loader & Dataset

SEQ_LEN     = 4       # sequence length
PATCH_SIZE  = 256     # size of random crop
BATCH_SIZE  = 4
EPOCHS      = 10

def load_multi(fp_seq):
    frames = []
    for fp in fp_seq[:SEQ_LEN]:
        with h5py.File(fp,'r') as f:
            cnt1, cnt2   = f['IMG_TIR1'][0][...], f['IMG_TIR2'][0][...]
            cnt_wv, cnt_mir = f['IMG_WV'][0][...], f['IMG_MIR'][0][...]
            cnt_vis      = f['IMG_VIS'][0][...]
            lut1, lut2   = f['IMG_TIR1_TEMP'][:],  f['IMG_TIR2_TEMP'][:]
            lut_wv, lut_mir = f['IMG_WV_TEMP'][:], f['IMG_MIR_TEMP'][:]
            lut_vis      = f['IMG_VIS_ALBEDO'][:]
        bt1 = lut1[cnt1]; bt2 = lut2[cnt2]
        wv  = lut_wv[cnt_wv]; mir = lut_mir[cnt_mir]
        vis = lut_vis[cnt_vis]
        frames.append(np.stack([bt1,bt2,wv,mir,vis],axis=-1)/300.0)
    X = np.stack(frames,axis=0).astype(np.float32)   # (T,H,W,C)

    with h5py.File(fp_seq[-1],'r') as f:
        cnt1, cnt2   = f['IMG_TIR1'][0][...], f['IMG_TIR2'][0][...]
        cnt_wv, cnt_mir = f['IMG_WV'][0][...], f['IMG_MIR'][0][...]
        lut1, lut2   = f['IMG_TIR1_TEMP'][:],  f['IMG_TIR2_TEMP'][:]
        lut_wv, lut_mir = f['IMG_WV_TEMP'][:], f['IMG_MIR_TEMP'][:]
    bt1_t = lut1[cnt1]; bt2_t = lut2[cnt2]
    wv_t  = lut_wv[cnt_wv]; mir_t = lut_mir[cnt_mir]

    # Example label generation identical to original
    cloud_mask  = (bt1 < 240).astype(np.float32)[None,...]   # dummy rule
    conv_mask   = (bt2 < 235).astype(np.float32)[None,...]
    fog_mask    = (wv  < 240).astype(np.float32)[None,...]
    moisture    = wv_t[...,None]/300.0
    thermo_contrast = (bt1_t-bt2_t)[...,None]/50.0
    temp_trend  = np.array([(bt1_t.mean()-bt1.mean())/10.0], dtype=np.float32)

    y = {
        'cloud'          : cloud_mask,
        'convective'     : conv_mask,
        'fog'            : fog_mask,
        'moisture'       : moisture,
        'thermo_contrast': thermo_contrast,
        'temp_trend'     : temp_trend
    }
    return X, y

def random_crop(X,y):
    H,W = X.shape[1], X.shape[2]
    i = np.random.randint(0, H-PATCH_SIZE)
    j = np.random.randint(0, W-PATCH_SIZE)
    Xc = X[:, i:i+PATCH_SIZE, j:j+PATCH_SIZE, :]
    yc = {}
    for k,v in y.items():
        if v.ndim==3:  # spatial map
            yc[k] = v[:, i:i+PATCH_SIZE, j:j+PATCH_SIZE]
        else:
            yc[k] = v
    return Xc, yc

class INSATSequenceDataset(Dataset):
    def __init__(self, seqs, transforms=True):
        self.seqs = seqs
        self.transforms = transforms
    def __len__(self):
        return len(self.seqs)
    def __getitem__(self, idx):
        X,y = load_multi(self.seqs[idx])
        if self.transforms:
            X,y = random_crop(X,y)
        # (T,H,W,C) -> (C,T,H,W) for ConvLSTM
        X = torch.from_numpy(X).permute(3,0,1,2)   # (C,T,H,W)
        y_tensors={}
        for k,v in y.items():
            y_tensors[k] = torch.from_numpy(v)
        return X, y_tensors


## Dataset Split & DataLoader Creation

In [12]:

# Define sequences (example: list of file paths or data sequences)
sequences = [f"data/file_{i}.h5" for i in range(100)]  # Replace with actual file paths or data

split       = int(0.9*len(sequences))
train_seqs  = sequences[:split]
val_seqs    = sequences[split:]

train_ds = INSATSequenceDataset(train_seqs, transforms=True)
val_ds   = INSATSequenceDataset(val_seqs, transforms=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=4, pin_memory=True)


## Model Definition (ConvLSTM + Multi‑Head Outputs)

In [13]:

# Cell 4: Model Definition & Compilation (PyTorch)

class ConvLSTMCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size, bias=True):
        super().__init__()
        padding = kernel_size//2
        self.input_channels  = input_channels
        self.hidden_channels = hidden_channels
        self.Gates = nn.Conv2d(in_channels=input_channels + hidden_channels,
                               out_channels=4 * hidden_channels,
                               kernel_size=kernel_size,
                               padding=padding,
                               bias=bias)
    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        combined = torch.cat([input_tensor, h_cur], dim=1)
        gates = self.Gates(combined)
        i,f,o,g = torch.chunk(gates, 4, dim=1)
        i = torch.sigmoid(i); f = torch.sigmoid(f)
        o = torch.sigmoid(o); g = torch.tanh(g)
        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, spatial_size, device):
        height, width = spatial_size
        h = torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
        c = torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
        return (h,c)

class ConvLSTM(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size, bias=True, num_layers=2):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_channels = hidden_channels if isinstance(hidden_channels,list) else [hidden_channels]*num_layers
        self.kernel_size = kernel_size if isinstance(kernel_size,list) else [kernel_size]*num_layers

        cells = []
        chs_in = input_channels
        for i in range(num_layers):
            cells.append(ConvLSTMCell(
                input_channels=chs_in,
                hidden_channels=self.hidden_channels[i],
                kernel_size=self.kernel_size[i],
                bias=bias
            ))
            chs_in = self.hidden_channels[i]
        self.cells = nn.ModuleList(cells)

    def forward(self, x):
        # x: (B,C,T,H,W)
        B,C,T,H,W = x.shape
        h,c = [],[]
        for i in range(self.num_layers):
            h.append(None); c.append(None)
        outputs = []
        for t in range(T):
            inp = x[:,:,t]
            for i,cell in enumerate(self.cells):
                if h[i] is None:
                    h[i], c[i] = cell.init_hidden(B, (H,W), x.device)
                h[i], c[i] = cell(inp, (h[i],c[i]))
                inp = h[i]
            outputs.append(h[-1])
        # return last hidden state
        return outputs[-1]

class FastMultitaskNowcast(nn.Module):
    def __init__(self):
        super().__init__()
        self.convlstm = ConvLSTM(input_channels=5, hidden_channels=[32,16], kernel_size=3, num_layers=2)
        # heads
        self.heads = nn.ModuleDict({
            'cloud': nn.Sequential(nn.Conv2d(16,1,1), nn.Sigmoid()),
            'convective': nn.Sequential(nn.Conv2d(16,1,1), nn.Sigmoid()),
            'fog': nn.Sequential(nn.Conv2d(16,1,1), nn.Sigmoid()),
            'moisture': nn.Conv2d(16,1,1),
            'thermo_contrast': nn.Conv2d(16,1,1),
        })
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        self.temp_head = nn.Linear(16,1)
    def forward(self, x):
        # x: (B,C,T,H,W)
        feat = self.convlstm(x)   # (B,16,H,W)
        out = {
            k: head(feat) for k,head in self.heads.items()
        }
        pooled = self.pool(feat).flatten(1)
        out['temp_trend'] = self.temp_head(pooled)
        return out

model = FastMultitaskNowcast().to(device)
print(model)


FastMultitaskNowcast(
  (convlstm): ConvLSTM(
    (cells): ModuleList(
      (0): ConvLSTMCell(
        (Gates): Conv2d(37, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1): ConvLSTMCell(
        (Gates): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (heads): ModuleDict(
    (cloud): Sequential(
      (0): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1))
      (1): Sigmoid()
    )
    (convective): Sequential(
      (0): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1))
      (1): Sigmoid()
    )
    (fog): Sequential(
      (0): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1))
      (1): Sigmoid()
    )
    (moisture): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1))
    (thermo_contrast): Conv2d(16, 1, kernel_size=(1, 1), stride=(1, 1))
  )
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (temp_head): Linear(in_features=16, out_features=1, bias=True)
)


## Training with Progress Bars

In [14]:

# Cell 5: Training Loop

loss_fns = {
    'cloud': nn.BCELoss(),
    'convective': nn.BCELoss(),
    'fog': nn.BCELoss(),
    'moisture': nn.MSELoss(),
    'thermo_contrast': nn.MSELoss(),
    'temp_trend': nn.MSELoss()
}
loss_weights = {'cloud':1,'convective':1,'fog':1,'moisture':0.5,'thermo_contrast':0.5,'temp_trend':0.1}

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

os.makedirs("checkpoints_3", exist_ok=True)

for epoch in range(1, EPOCHS+1):
    model.train()
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    running_loss = 0.0
    for Xb, yb in pbar:
        Xb = Xb.to(device)
        yb = {k:v.to(device) for k,v in yb.items()}
        optimizer.zero_grad()
        with autocast():
            preds = model(Xb)
            losses = []
            for k in preds:
                pred = preds[k]
                target = yb[k]
                if pred.ndim==4 and target.ndim==4:
                    pred = pred.squeeze(1)
                    target = target.squeeze(1)
                loss = loss_fns[k](pred.float(), target.float())
                losses.append(loss * loss_weights[k])
            loss_total = sum(losses)
        scaler.scale(loss_total).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss_total.item()
        pbar.set_postfix(loss=running_loss/ (pbar.n+1))
    # ── validation ----------------------------------------------------------------
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb = Xb.to(device)
            yb = {k:v.to(device) for k,v in yb.items()}
            with autocast():
                preds = model(Xb)
                losses = []
                for k in preds:
                    pred = preds[k]
                    target = yb[k]
                    if pred.ndim==4 and target.ndim==4:
                        pred = pred.squeeze(1)
                        target = target.squeeze(1)
                    loss = loss_fns[k](pred.float(), target.float())
                    losses.append(loss * loss_weights[k])
                val_loss += sum(losses).item()
    val_loss /= len(val_loader)
    print(f"Validation loss: {val_loss:.4f}")
    torch.save(model.state_dict(), f"checkpoints_3/model_epoch_{epoch:02d}.pth")


Epoch 1/10:   0%|          | 0/23 [00:05<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 25348, 8436, 20972, 1952) exited unexpectedly

## Evaluation & Metrics Display

In [ ]:

# Cell 6 — Evaluation
seg_keys = ["cloud","convective","fog"]
reg_keys = ["moisture","thermo_contrast","temp_trend"]

y_true_seg, y_pred_seg = {k:[] for k in seg_keys}, {k:[] for k in seg_keys}
y_true_reg, y_pred_reg = {k:[] for k in reg_keys}, {k:[] for k in reg_keys}

model.eval()
with torch.no_grad():
    for Xb, yb in tqdm(val_loader, desc="Evaluation"):
        Xb = Xb.to(device)
        preds = model(Xb)
        for k in seg_keys:
            truth = yb[k].view(-1).numpy()
            pred  = (preds[k].cpu().view(-1) > 0.5).numpy()
            y_true_seg[k].append(truth)
            y_pred_seg[k].append(pred)
        for k in reg_keys:
            truth = yb[k].view(-1).numpy()
            pred  = preds[k].cpu().view(-1).numpy()
            y_true_reg[k].append(truth)
            y_pred_reg[k].append(pred)

for d in (y_true_seg, y_pred_seg, y_true_reg, y_pred_reg):
    for k in d: d[k] = np.concatenate(d[k])

# ── segmentation --------------------------------------------------------------
import pandas as pd
seg_rows=[]
for k in seg_keys:
    cm = confusion_matrix(y_true_seg[k], y_pred_seg[k])
    seg_rows.append({
        "Task":k,
        "Accuracy": accuracy_score(y_true_seg[k], y_pred_seg[k]),
        "Precision": precision_score(y_true_seg[k], y_pred_seg[k]),
        "Recall": recall_score(y_true_seg[k], y_pred_seg[k]),
        "F1": f1_score(y_true_seg[k], y_pred_seg[k])
    })
df_seg = pd.DataFrame(seg_rows).set_index("Task")
print("### Segmentation"); display(df_seg)

# ── regression ---------------------------------------------------------------
reg_rows=[]
for k in reg_keys:
    reg_rows.append({
        "Task":k,
        "MSE": mean_squared_error(y_true_reg[k], y_pred_reg[k]),
        "MAE": mean_absolute_error(y_true_reg[k], y_pred_reg[k])
    })
df_reg = pd.DataFrame(reg_rows).set_index("Task")
print("### Regression"); display(df_reg)

# ── confusion matrices --------------------------------------------------------
for k in seg_keys:
    cm = confusion_matrix(y_true_seg[k], y_pred_seg[k])
    plt.figure(figsize=(4,4))
    plt.title(f"{k.capitalize()} – Confusion matrix")
    plt.imshow(cm, cmap="Blues"); plt.xlabel("Pred"); plt.ylabel("True")
    for (i,j),v in np.ndenumerate(cm): plt.text(j,i,str(v), ha="center", va="center")
    plt.colorbar(); plt.show()
